In [29]:
import os
import glob
import pickle
import numpy as np
def parse_logfile(filename):
    logfile = open(filename, 'r')
    lines = logfile.readlines()
    logfile.close()

    iteration_data = []
    current_iteration = {}

    for line in lines:
        if line.startswith('ITERATION:'):
            if current_iteration:
                iteration_data.append(current_iteration)
                current_iteration = {}
        elif line.startswith('summary'):
            current_iteration['summary'] = line.strip()
        elif line.startswith('number of regions step'):
            current_iteration['number_of_regions_step'] = int(line.split()[-1])
        elif line.startswith('number of regions total'):
            current_iteration['number_of_regions_total'] = int(line.split()[-1])
        elif line.startswith('tstep'):
            a = line.replace(',', '')
            current_iteration['tstep'] = float(a.split()[1])
            current_iteration['t_total'] = float(a.split()[3])
        elif line.startswith('tsample'):
            a = line.replace(',', '')
            current_iteration['tsample'] = float(a.split()[1])
            current_iteration['t_visgraph'] = float(a.split()[3])
            current_iteration['t_mhs'] = float(a.split()[5])
            current_iteration['t_regions'] = float(a.split()[7])

        # elif line.startswith('t_mhs'):
        #     current_iteration['t_mhs'] = float(line.split()[1])
        # elif line.startswith('t_regions'):
        #     current_iteration['t_regions'] = float(line.split()[1])
        elif line.startswith('coverage'):
            current_iteration['coverage'] = float(line.split()[1])
        elif '[VisSeeder] Coverage met, terminated' in line:
            current_iteration['termination'] = line.strip()

    if current_iteration:
        iteration_data.append(current_iteration)
    return iteration_data


logs_directory = 'logs'

# Get a list of all subdirectories in the logs directory
experiment_directories = [directory for directory in glob.glob(os.path.join(logs_directory, '*')) if os.path.isdir(directory)]

# Define a dictionary to store the grouped experiments
grouped_experiments = {}

# Iterate over each experiment directory
for experiment_directory in experiment_directories:
    # Parse the experiment name
    experiment_name = os.path.basename(experiment_directory)
    name_parts = experiment_name.split('_')
    
    # Extract the relevant values from the experiment name
    if '3dof' in experiment_name and not 'cvx_hull' in experiment_name:
        b = 1  #int(name_parts[-5])
        # print(experiment_name)
    if '3dof_flipper2' in experiment_name and not 'cvx_hull' in experiment_name:
        b = 6  #int(name_parts[-5])
        # print(experiment_name)
    elif '5dof' in experiment_name and 'greedy' in experiment_name:
        b = 2
    elif '7dof_iiwa' in experiment_name and 'greedy' in experiment_name:
        b = 3
    else:
        continue
    N = int(name_parts[-2])
    
    # Get the last pickle file in the 'data' subdirectory
    data_directory = os.path.join(experiment_directory, 'data')
    pickle_files = glob.glob(os.path.join(data_directory, '*.pkl'))
    pkl_files = os.listdir(data_directory)
    pkl_idx = np.argsort([int(s.replace('it_', '').replace('.pkl', '')) for s in pkl_files])
    
    #pickle_files.sort()
    
    last_pickle_file = data_directory+'/'+pkl_files[pkl_idx[-1]]#pickle_files[-1]#max(pickle_files, key=os.path.getctime)
    
    # Load the last pickle file
    with open(last_pickle_file, 'rb') as file:
        data = pickle.load(file)
    
    # Find the 'summar' subdirectory and get the path to the text file
    summar_directory = os.path.join(experiment_directory, 'summary')
    text_files = glob.glob(os.path.join(summar_directory, '*.txt'))
    last_text_file = max(text_files, key=os.path.getctime)
    

    iteration_dictionaries = parse_logfile(last_text_file)
    tsample = np.sum([i['tsample'] for i in iteration_dictionaries[1:]])
    tvis = np.sum([i['t_visgraph'] for i in iteration_dictionaries[1:]])
    tmhs = np.sum([i['t_mhs'] for i in iteration_dictionaries[1:]])
    tregions = np.sum([i['t_regions'] for i in iteration_dictionaries[1:]])
    coverage = np.sum([i['coverage'] for i in iteration_dictionaries[1:]])
    # # Read the last line of the text file to extract coverage
    # with open(last_text_file, 'r') as file:
    #     lines = file.readlines()
    #     last_line = lines[-1].strip()
    #     coverage = float(last_line.split()[1])

    # Group experiments by 'b' value
    if b not in grouped_experiments:
        grouped_experiments[b] = {}
    
    # Group experiments by 'N' value
    if N not in grouped_experiments[b]:
        grouped_experiments[b][N] = []
    
    # Add the experiment data to the grouped experiments dictionary
    grouped_experiments[b][N].append([
            np.sum(len(g) for g in data['rb']), coverage , data['ttotal'], tsample, tvis, tmhs, tregions])

# Print the grouped experiments
for b, experiments in grouped_experiments.items():
    print(f"Experiments with b = {b}:")
    for N, experiment_data in experiments.items():
        print(f"  Experiments with N = {N}:")
        for dat in experiment_data:
            print(f"    Experiment data: {dat}")

Experiments with b = 2:
  Experiments with N = 1000:
    Experiment data: [38, 1.4798, 669.254762172699, 4.333, 39.987, 439.079, 185.857]
    Experiment data: [36, 1.4496, 544.01407122612, 4.037, 40.953, 306.65, 192.374]
    Experiment data: [35, 1.4409999999999998, 469.2104163169861, 3.931, 42.474999999999994, 242.69, 180.11599999999999]
    Experiment data: [36, 1.4818, 554.8998708724976, 4.613, 40.102000000000004, 329.172, 181.01299999999998]
    Experiment data: [36, 1.4694, 616.4663352966309, 4.203, 39.327, 402.71999999999997, 170.215]
    Experiment data: [36, 1.4582, 486.2716007232666, 4.125, 40.04, 265.565, 176.542]
    Experiment data: [34, 1.4436, 542.9091618061066, 3.9939999999999998, 41.372, 327.405, 170.138]
    Experiment data: [33, 1.4180000000000001, 481.6702198982239, 3.8970000000000002, 39.925, 256.695, 181.154]
    Experiment data: [32, 1.42, 647.4932613372803, 3.878, 39.385000000000005, 419.081, 185.149]
    Experiment data: [35, 1.4542000000000002, 522.378255128860

<ipython-input-29-e17e2b9265f6>:120: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  np.sum(len(g) for g in data['rb']), coverage , data['ttotal'], tsample, tvis, tmhs, tregions])


In [30]:
experiment_name

'experiment_7dof_iiwa__1_1500_0.300greedy20230908201834'

In [31]:
data = np.array(grouped_experiments[3][1500])
means = data.mean(axis =0)
std = data.std(axis =0)

print(f"mean: regions {means[0]:.1f} time: {means[2]:.1f}")
print(f"std: regions {std[0]:.1f} time: {std[2]:.1f}")

mean: regions 46.3 time: 5726.3
std: regions 4.5 time: 1737.8


In [47]:
import yaml
experiment_directories_red = []
for e in experiment_directories:
    if '7dof' in e and 'greedy' in e:
        experiment_directories_red.append(e)
experiment_directory = experiment_directories_red[1]
data_directory = os.path.join(experiment_directory, 'data')
pickle_files = glob.glob(os.path.join(data_directory, '*.pkl'))
pkl_files = os.listdir(data_directory)
pkl_idx = np.argsort([int(s.replace('it_', '').replace('.pkl', '')) for s in pkl_files])

#pickle_files.sort()

last_pickle_file = data_directory+'/'+pkl_files[pkl_idx[-1]]#pickle_files[-1]#max(pickle_files, key=os.path.getctime)

# Load the last pickle file
with open(last_pickle_file, 'rb') as file:
    data = pickle.load(file)
# print(experiment_directory)
# with open(experiment_directory+'/cfg.yaml', 'r') as f:
#     data  = yaml.load(f)


In [48]:
# vgraph_verts = data['vg']
# vgraph_adj_mats= data['vad']

In [49]:
from pydrake.all import MathematicalProgram, Solve, SolverOptions, CommonSolverOption

def max_clique(adj_mat):
    #assert adj_mat.shape[0] == len(graph_vertices)
    #assert graph_vertices[0, :].shape[0] == points_to_exclude.shape[1]
    
    #dim = graph_vertices.shape[1]
    #compute radius of circumscribed sphere of all points to get soft margin size
    n = adj_mat.shape[0]
    
    prog = MathematicalProgram()
    v = prog.NewBinaryVariables(n)
    prog.AddLinearCost(-np.sum(v))
    
    
    for i in range(0,n):
        for j in range(i+1,n):
            if adj_mat[i,j] == 0:
                prog.AddLinearConstraint(v[i] + v[j] <= 1)

    solver_options = SolverOptions()
    solver_options.SetOption(CommonSolverOption.kPrintToConsole, 1)

    result = Solve(prog, solver_options=solver_options)
    print(result.is_success())
    return np.where(result.GetSolution(v)==1)[0]

In [50]:
print(experiment_directory.replace('logs/experiment_',''))
from clique_covers import extend_cliques, compute_greedy_clique_partition
clique_sizes = []
num_vertex_holes_in_clique = []

for it, (verts, admat) in enumerate(zip(data['vg'], data['vad'])):
    cliques = compute_greedy_clique_partition(admat.toarray(), 10)
    cliques = extend_cliques(admat.toarray(), cliques)
    for clique in cliques:
        #verts = vgraph_verts[0]
        num_non_clique_verts = verts.shape[0]-len(clique)
        non_clique_idx = np.delete(np.arange(len(verts)), clique)
        clique_verts = verts[clique]
        non_clique_verts = verts[non_clique_idx]
        #def convex hull program
        n = 0
        for vert_of_interest in non_clique_verts:
            prog = MathematicalProgram()
            num_non_clique_verts = 1

            lam = prog.NewContinuousVariables(num_non_clique_verts, len(clique))
            A = np.ones((1,len(clique)))
            for i in range(num_non_clique_verts):
                prog.AddLinearConstraint((A@lam[i,:])[0] == 1)
            for l in lam.flatten():
                prog.AddLinearConstraint(l>=0)
            #pred = lam@verts[clique]
            for i in range(num_non_clique_verts):
                prog.AddLinearEqualityConstraint(clique_verts.T, vert_of_interest.reshape(-1,1), lam[i,:])

            #verts[non_clique_idx[i],:].reshape(-1,1)

            result = Solve(prog)
            #print(result.is_success())
            if result.is_success():
                n+=1
        clique_sizes.append(len(clique))
        num_vertex_holes_in_clique.append(n)
        print(f"CBS Iteration {it}")
        print(f"Max clique on visibility graph has size {len(clique)}")
        print(f"Number of non-clique members lying in the convex hull of the clique: {n}")
        #print(f"number of verts in the convex hull of maxclique that are not in the maxclique: {n}, for clique of size {len(clique)}")



7dof_iiwa__34_1500_0.300greedy20230910054858


/usr/lib/python3/dist-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
<ipython-input-50-182e59b09b09>:24: RuntimeWarning: invalid value encountered in matmul
  prog.AddLinearConstraint((A@lam[i,:])[0] == 1)


CBS Iteration 0
Max clique on visibility graph has size 483
Number of non-clique members lying in the convex hull of the clique: 0
CBS Iteration 0
Max clique on visibility graph has size 238
Number of non-clique members lying in the convex hull of the clique: 0
CBS Iteration 0
Max clique on visibility graph has size 250
Number of non-clique members lying in the convex hull of the clique: 0
CBS Iteration 0
Max clique on visibility graph has size 294
Number of non-clique members lying in the convex hull of the clique: 0
CBS Iteration 0
Max clique on visibility graph has size 348
Number of non-clique members lying in the convex hull of the clique: 0
CBS Iteration 0
Max clique on visibility graph has size 242
Number of non-clique members lying in the convex hull of the clique: 0
CBS Iteration 0
Max clique on visibility graph has size 149
Number of non-clique members lying in the convex hull of the clique: 0
CBS Iteration 0
Max clique on visibility graph has size 164
Number of non-clique me

In [51]:
print(f"number of cliques {len(clique_sizes)}")
print(f"number of violating cliques {len(np.where(np.array(num_vertex_holes_in_clique)>0)[0])}")
ioi = np.where(np.array(num_vertex_holes_in_clique)>0)[0]
sizes_oi = [c for c in np.array(clique_sizes)[ioi]]
numvholes_in_clique_oi = [c for c in np.array(num_vertex_holes_in_clique)[ioi]]
strings = [f" {n}/{s} " for n,s in zip(numvholes_in_clique_oi, sizes_oi)]
s = ""
for a in strings:
    s+=a
print(f"violators (#holes/cliquesize): {s}")

number of cliques 67
number of violating cliques 1
violators (#holes/cliquesize):  1/113 


In [45]:
print(f"number of cliques {len(clique_sizes)}")
print(f"number of violating cliques {len(np.where(np.array(num_vertex_holes_in_clique)>0)[0])}")
ioi = np.where(np.array(num_vertex_holes_in_clique)>0)[0]
sizes_oi = [c for c in np.array(clique_sizes)[ioi]]
numvholes_in_clique_oi = [c for c in np.array(num_vertex_holes_in_clique)[ioi]]
strings = [f" {n}/{s} " for n,s in zip(numvholes_in_clique_oi, sizes_oi)]
s = ""
for a in strings:
    s+=a
print(f"violators (#holes/cliquesize): {s}")

number of cliques 38
number of violating cliques 5
violators (#holes/cliquesize):  1/74  4/173  1/279  1/306  9/188 


In [35]:
print(num_vertex_holes_in_clique)
print([len(c) for c in cliques])

[0, 0, 0, 0, 1, 0, 0, 0, 4, 1, 0, 1, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[119, 102, 96, 97, 59, 91, 68, 41, 85, 31, 60, 44, 85, 32, 84, 15, 65, 21, 42, 41, 48, 54, 74]


In [25]:
num_vertex_holes_in_clique

[0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 4,
 1,
 0,
 1,
 0,
 9,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [11]:
(A@lam[i,:])[0]

<Expression "(x(0,0) + x(0,1) + x(0,2) + x(0,3) + x(0,4) + x(0,5) + x(0,6) + x(0,7) + x(0,8) + x(0,9) + x(0,10) + x(0,11) + x(0,12) + x(0,13) + x(0,14) + x(0,15) + x(0,16) + x(0,17) + x(0,18) + x(0,19) + x(0,20) + x(0,21) + x(0,22) + x(0,23) + x(0,24) + x(0,25) + x(0,26) + x(0,27) + x(0,28) + x(0,29) + x(0,30) + x(0,31) + x(0,32) + x(0,33) + x(0,34) + x(0,35) + x(0,36) + x(0,37) + x(0,38) + x(0,39) + x(0,40) + x(0,41) + x(0,42) + x(0,43) + x(0,44) + x(0,45) + x(0,46) + x(0,47) + x(0,48) + x(0,49) + x(0,50) + x(0,51) + x(0,52) + x(0,53) + x(0,54) + x(0,55) + x(0,56) + x(0,57) + x(0,58) + x(0,59) + x(0,60) + x(0,61) + x(0,62) + x(0,63) + x(0,64) + x(0,65) + x(0,66) + x(0,67) + x(0,68) + x(0,69) + x(0,70) + x(0,71) + x(0,72) + x(0,73) + x(0,74) + x(0,75) + x(0,76) + x(0,77) + x(0,78) + x(0,79) + x(0,80) + x(0,81) + x(0,82) + x(0,83) + x(0,84) + x(0,85) + x(0,86) + x(0,87) + x(0,88) + x(0,89) + x(0,90) + x(0,91) + x(0,92) + x(0,93) + x(0,94) + x(0,95) + x(0,96) + x(0,97) + x(0,98) + x(0,99

In [7]:
data = np.array(grouped_experiments[1][1])
means = data.mean(axis =0)
std = data.std(axis =0)

print(f"mean: regions {means[0]:.1f} time: {means[2]:.1f}")
print(f"std: regions {std[0]:.1f} time: {std[2]:.1f}")

mean: regions 10.4 time: 203.4
std: regions 1.9 time: 29.0


In [5]:
'cvx_hull' in experiment_name

True

In [ ]:
import os
import glob
import pickle
import numpy as np
def parse_logfile(filename):
    logfile = open(filename, 'r')
    lines = logfile.readlines()
    logfile.close()

    iteration_data = []
    current_iteration = {}

    for line in lines:
        if line.startswith('ITERATION:'):
            if current_iteration:
                iteration_data.append(current_iteration)
                current_iteration = {}
        elif line.startswith('summary'):
            current_iteration['summary'] = line.strip()
        elif line.startswith('number of regions step'):
            current_iteration['number_of_regions_step'] = int(line.split()[-1])
        elif line.startswith('number of regions total'):
            current_iteration['number_of_regions_total'] = int(line.split()[-1])
        elif line.startswith('tstep'):
            a = line.replace(',', '')
            current_iteration['tstep'] = float(a.split()[1])
            current_iteration['t_total'] = float(a.split()[3])
        elif line.startswith('tsample'):
            a = line.replace(',', '')
            current_iteration['tsample'] = float(a.split()[1])
            current_iteration['t_visgraph'] = float(a.split()[3])
        elif line.startswith('t_mhs'):
            current_iteration['t_mhs'] = float(line.split()[1])
        elif line.startswith('t_regions'):
            current_iteration['t_regions'] = float(line.split()[1])
        elif line.startswith('coverage'):
            current_iteration['coverage'] = float(line.split()[1])
        elif '[VisSeeder] Coverage met, terminated' in line:
            current_iteration['termination'] = line.strip()

    if current_iteration:
        iteration_data.append(current_iteration)
    return iteration_data


logs_directory = '../logs'

# Get a list of all subdirectories in the logs directory
experiment_directories = [directory for directory in glob.glob(os.path.join(logs_directory, '*')) if os.path.isdir(directory)]

# Define a dictionary to store the grouped experiments
grouped_experiments = {}

# Iterate over each experiment directory
for experiment_directory in experiment_directories:
    # Parse the experiment name
    experiment_name = os.path.basename(experiment_directory)
    name_parts = experiment_name.split('_')
    
    # Extract the relevant values from the experiment name
    if '5dof_ur_naive' in experiment_name:
        b = 1  #int(name_parts[-5])
    else:
        continue
    N = int(name_parts[-3])
    
    # Get the last pickle file in the 'data' subdirectory
    data_directory = os.path.join(experiment_directory, 'data')
    pickle_files = glob.glob(os.path.join(data_directory, '*.pkl'))
    pkl_files = os.listdir(data_directory)
    pkl_idx = np.argsort([int(s.replace('it_', '').replace('.pkl', '')) for s in pkl_files])
    
    #pickle_files.sort()
    
    last_pickle_file = data_directory+'/'+pkl_files[pkl_idx[-1]]#pickle_files[-1]#max(pickle_files, key=os.path.getctime)
    
    # Load the last pickle file
    with open(last_pickle_file, 'rb') as file:
        data = pickle.load(file)
    
    # Find the 'summar' subdirectory and get the path to the text file
    summar_directory = os.path.join(experiment_directory, 'summary')
    text_files = glob.glob(os.path.join(summar_directory, '*.txt'))
    last_text_file = max(text_files, key=os.path.getctime)
    

    iteration_dictionaries = parse_logfile(last_text_file)
    tsample = np.sum([i['tsample'] for i in iteration_dictionaries[1:]])
    tvis = np.sum([i['t_visgraph'] for i in iteration_dictionaries[1:]])
    tmhs = np.sum([i['t_mhs'] for i in iteration_dictionaries[1:]])
    tregions = np.sum([i['t_regions'] for i in iteration_dictionaries[1:]])
    # # Read the last line of the text file to extract coverage
    # with open(last_text_file, 'r') as file:
    #     lines = file.readlines()
    #     last_line = lines[-1].strip()
    #     coverage = float(last_line.split()[1])

    # Group experiments by 'b' value
    if b not in grouped_experiments:
        grouped_experiments[b] = {}
    
    # Group experiments by 'N' value
    if N not in grouped_experiments[b]:
        grouped_experiments[b][N] = []
    
    # Add the experiment data to the grouped experiments dictionary
    grouped_experiments[b][N].append([
            np.sum(len(g) for g in data['rb']), data['cov'] , data['ttotal'], tsample, tvis, tmhs, tregions])

# Print the grouped experiments
for b, experiments in grouped_experiments.items():
    print(f"Experiments with b = {b}:")
    for N, experiment_data in experiments.items():
        print(f"  Experiments with N = {N}:")
        for dat in experiment_data:
            print(f"    Experiment data: {dat}")

In [ ]:
data = np.array(grouped_experiments[1][1])
means = data.mean(axis =0)
std = data.std(axis =0)

print(f"mean: regions {means[0]:.1f} time: {means[2]:.1f}")
print(f"std: regions {std[0]:.1f} time: {std[2]:.1f}")

In [81]:
# bernoulli test
from scipy.special import comb
from scipy.stats import norm
import numpy as np
alpha_0 = 0.8
alpha_z = 0.05
n = 5000
names = ['Village', '3dof', '5dof', '7dof']
cov_thr = [0.8, 0.9, 0.75, 0.7]
print(f"Number of samples used in all tests {n}")
print(f"#######################################################")

for na, alpha_0 in zip(names,cov_thr):
    #norm_approx_cdf(0.75*n,n, alpha_0)
    mu = n*alpha_0
    sigma= np.sqrt(n*alpha_0*(1-alpha_0))
    #print(sum)
    #twosided
    top =norm.ppf(0.025, loc = mu, scale= sigma)/n
    bottom = norm.ppf(0.975, loc = mu, scale= sigma)/n 
    print(f"Experiment {na}")
    print(f"Two sided bernoulli test 95% confidence coverage lies between {bottom:.3f} and {top:.3f}")
    #onesided
    print(f"One sided bernoulli test: 95% confidence coverage lies above {norm.ppf(0.05, loc = mu, scale= sigma)/n:.3f} ")


Number of samples used in all tests 5000
#######################################################
Experiment Village
Two sided bernoulli test 95% confidence coverage lies between 0.811 and 0.789
One sided bernoulli test: 95% confidence coverage lies above 0.791 
Experiment 3dof
Two sided bernoulli test 95% confidence coverage lies between 0.908 and 0.892
One sided bernoulli test: 95% confidence coverage lies above 0.893 
Experiment 5dof
Two sided bernoulli test 95% confidence coverage lies between 0.762 and 0.738
One sided bernoulli test: 95% confidence coverage lies above 0.740 
Experiment 7dof
Two sided bernoulli test 95% confidence coverage lies between 0.713 and 0.687
One sided bernoulli test: 95% confidence coverage lies above 0.689 


In [80]:
alpha_0 = 0.8
n = 5000
mu = n*alpha_0
sigma= np.sqrt(n*alpha_0*(1-alpha_0))
norm.ppf(0.975, loc = mu, scale = sigma)/n

0.8110872305947975

In [67]:
norm.ppf(0.025, loc = mu, scale= sigma)

3436.489908189888

In [5]:
np.int(np.ceil(alpha*n))

9000

In [6]:
range(np.int(np.ceil(alpha*n)), n+1)

TypeError: 'float' object cannot be interpreted as an integer

In [7]:
n

10000.0